<a href="https://colab.research.google.com/github/1eetnals/News-group-classification/blob/main/TfidfVectorizer_%2B_MLPClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/dacon/뉴스 그룹 분류 경진대회/train.csv")
test = pd.read_csv("/content/drive/MyDrive/dacon/뉴스 그룹 분류 경진대회/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/dacon/뉴스 그룹 분류 경진대회/sample_submission.csv")

In [ ]:
# 결측치 존재 여부를 확인해주는 함수
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


In [ ]:
train.head()

,id,text,target
0,0,"\nThey were, and even if Washington might cons...",10
1,1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,2,\n\n\nNot to worry. The Masons have been demo...,19
3,3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,4,Help: I am running some sample problems from O...,5


In [ ]:
test.head()

,id,text
0,0,\nThe VL-IDE Adapter can be much faster then t...
1,1,\n\nYeah. In a fire that reportedly burned ho...
2,2,":Judge: ""I grant you immunity from whatever ma..."
3,3,"I, too, put a corbin seat on my Hawk. I got t..."
4,4,\n\nDo I ever!!!!!! After 2 years of having h...


In [ ]:
submission.head()

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13


In [ ]:
import re 

def clean_text(texts): 
  corpus = [] 
  for i in range(0, len(texts)): 

    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
    #review = re.sub(r'\d+','', review)#숫자 제거
    #review = review.lower() #소문자 변환
    review = re.sub(r'\s+', ' ', review) #extra space 제거
    review = re.sub(r'<[^>]+>','',review) #Html tags 제거
    review = re.sub(r'\s+', ' ', review) #spaces 제거
    review = re.sub(r"^\s+", '', review) #space from start 제거
    review = re.sub(r'\s+$', '', review) #space from the end 제거
    review = re.sub(r'_', ' ', review) #space from the end 제거
    corpus.append(review) 
  
  return corpus

In [ ]:
temp = clean_text(train['text']) #메소드 적용
train['text'] = temp
train.head()

,id,text,target
0,0,They were and even if Washington might conside...,10
1,1,We run SpaceNews Views on our STAREACH BBS a l...,14
2,2,Not to worry The Masons have been demonized an...,19
3,3,Only Brendan McKay or maybe ARF would come to ...,17
4,4,Help I am running some sample problems from OR...,5


In [ ]:
temp = clean_text(test['text']) #메소드 적용
test['text'] = temp
test.head()

,id,text
0,0,The VLIDE Adapter can be much faster then the ...
1,1,Yeah In a fire that reportedly burned hotter t...
2,2,Judge I grant you immunity from whatever may b...
3,3,I too put a corbin seat on my Hawk I got the s...
4,4,Do I ever After 2 years of having health probl...


### CountVectorizer

In [ ]:
X = train.text #training 데이터에서 문서 추출
y = train.target #training 데이터에서 라벨 추출

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #sklearn 패키지의 CountVectorizer import

sample_vectorizer = CountVectorizer() #객체 생성

In [ ]:
vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X) #countvectorizer 학습
X = vectorizer.transform(X) #transform

### TfidVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2),min_df = 0.0005)
#vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)


vectorizer.fit(np.array(train["text"]))

train_vec = vectorizer.transform(train["text"])
train_y = train["target"]

test_vec = vectorizer.transform(test["text"])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(train_vec,train_y,random_state=0,test_size=0.2)

In [ ]:
train_vec.shape

(9233, 45056)

### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(x_train, y_train) #모델 학습

LogisticRegression(max_iter=500)

In [ ]:
model.score(x_val, y_val)

0.6762317271250676

### SVM

In [ ]:
from sklearn import svm

In [ ]:
model = svm.SVC(kernel='linear')
model.fit(x_train, y_train)


SVC(kernel='linear')

In [ ]:
model.score(x_val, y_val)

0.7005955603681646

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model = LinearSVC()
model.fit(train_vec, train_y)

LinearSVC()

In [ ]:
model.score(x_val, y_val)

0.9891716296697347

### SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier


In [ ]:
model = SGDClassifier()
model.fit(train_vec, train_y)

SGDClassifier()

In [ ]:
model.score(x_val, y_val)

0.9848402815376286

#### kfold

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
model = SGDClassifier()
kfold = KFold(n_splits=5)
cv_accuracy = [] # 폴드 세트별 정확도를 담을 리스트
print('데이터 세트 크기: ', train_vec.shape[0])

n_iter = 0

for train_index, test_index in kfold.split(train_vec):
    X_train, X_test = train_vec[train_index], train_vec[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]
    
    # 학습, 예측
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    n_iter += 1
    
    # 정확도 계산
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
    print('\n## 평균 검증 정확도: ', np.mean(cv_accuracy))

데이터 세트 크기:  9233

#1 교차 검증 정확도 : 0.6816, 학습 데이터 크기: 7386, 검증 데이터 크기: 1847
#1 검증 세트 인덱스: [   0    1    2 ... 1844 1845 1846]

## 평균 검증 정확도:  0.6816

#2 교차 검증 정확도 : 0.6881, 학습 데이터 크기: 7386, 검증 데이터 크기: 1847
#2 검증 세트 인덱스: [1847 1848 1849 ... 3691 3692 3693]

## 평균 검증 정확도:  0.68485

#3 교차 검증 정확도 : 0.6827, 학습 데이터 크기: 7386, 검증 데이터 크기: 1847
#3 검증 세트 인덱스: [3694 3695 3696 ... 5538 5539 5540]

## 평균 검증 정확도:  0.6841333333333334

#4 교차 검증 정확도 : 0.6977, 학습 데이터 크기: 7387, 검증 데이터 크기: 1846
#4 검증 세트 인덱스: [5541 5542 5543 ... 7384 7385 7386]

## 평균 검증 정확도:  0.6875249999999999

#5 교차 검증 정확도 : 0.7021, 학습 데이터 크기: 7387, 검증 데이터 크기: 1846
#5 검증 세트 인덱스: [7387 7388 7389 ... 9230 9231 9232]

## 평균 검증 정확도:  0.6904399999999999


####GridSearchCV


In [ ]:
from pandas.core.dtypes.cast import maybe_unbox_datetimelike
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(train_vec,train_y, test_size = 0.2, random_state=121)
dtree = SGDClassifier()

# 파라미터는 딕셔너리 형태로 입력
#parameters = {'max_depth': [1, 2, 3], 'min_samples_split' : [2, 3] }
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

#model = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)
model = GridSearchCV(dtree,param_grid=param_grid)


model.fit(X_train, y_train)

scores_df = pd.DataFrame(model.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'alpha': 0.0001, 'average': True, 'l1_ratio':...",0.712698,2,0.723275,0.689912,0.695328
1,"{'alpha': 0.0001, 'average': True, 'l1_ratio':...",0.710802,11,0.725304,0.688558,0.693974
2,"{'alpha': 0.0001, 'average': True, 'l1_ratio':...",0.712968,1,0.728687,0.690589,0.697360
3,"{'alpha': 0.0001, 'average': True, 'l1_ratio':...",0.709855,17,0.721245,0.689235,0.691943
4,"{'alpha': 0.0001, 'average': True, 'l1_ratio':...",0.711479,8,0.728011,0.691943,0.694651
...,...,...,...,...,...,...
95,"{'alpha': 1.0, 'average': False, 'l1_ratio': 0...",0.065667,86,0.052097,0.104265,0.058903
96,"{'alpha': 1.0, 'average': False, 'l1_ratio': 0...",0.071894,84,0.064276,0.128639,0.052133
97,"{'alpha': 1.0, 'average': False, 'l1_ratio': 0...",0.064040,90,0.063599,0.054164,0.052133
98,"{'alpha': 1.0, 'average': False, 'l1_ratio': 0...",0.047524,100,0.034506,0.054164,0.054164


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer #sklearn 패키지의 CountVectorizer import
from sklearn.model_selection import cross_validate



In [ ]:
text_clf = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', 
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),])


In [ ]:
from sklearn.model_selection import train_test_split
X = train.text #training 데이터에서 문서 추출
y = train.target #training 데이터에서 라벨 추출
x_train,x_val,y_train,y_val=train_test_split(X,y,random_state=0,test_size=0.2)

In [ ]:
pred = text_clf.predict(x_val)

In [ ]:
np.mean(pred == y_val)

In [ ]:
cross_validate(text_clf,X,y, cv=KFold(3,shuffle=True, random_state=23), return_train_score=True)

{'fit_time': array([1.20959735, 1.14576006, 1.11075854]),
 'score_time': array([0.40679097, 0.44181705, 0.46764612]),
 'test_score': array([0.72904483, 0.71410006, 0.71335717]),
 'train_score': array([0.95320877, 0.95662063, 0.95760234])}

### Multinominal naive bayes



In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model=MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [ ]:
model.score(x_val, y_val)

0.5300487276664861

### MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
import torch
model = MLPClassifier()
model.fit(train_vec, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier()

### 확인

In [ ]:
from sklearn.metrics import accuracy_score

#run model
y_pred = model.predict(train_vec[2])
print('예측 라벨 : ', y_pred)
print('실제 라벨 : ', train.target[2])

예측 라벨 :  [19]
실제 라벨 :  19


### test

In [ ]:
pred = model.predict(test_vec)

In [ ]:
submission["target"] = pred

In [ ]:
submission

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13
...,...,...
9228,9228,16
9229,9229,1
9230,9230,4
9231,9231,0


In [ ]:
submission.to_csv("/content/drive/MyDrive/dacon/뉴스 그룹 분류 경진대회/submission9.csv", index = False)